In [1]:
import pandas as pd

train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

train.head()

ID       x_0       x_1       x_2       x_3       x_4       x_5  \
0  TRAIN_00000  1.006187 -1.962566  1.247535  0.926500 -0.265766 -1.789301   
1  TRAIN_00001  1.024647 -2.472625  1.144386  0.846499 -0.287336 -1.756679   
2  TRAIN_00002  1.062444 -2.451003  1.186546  0.873599 -0.257828 -1.802735   
3  TRAIN_00003  1.089189 -2.458470  1.184531  0.810867 -0.276517 -1.787739   
4  TRAIN_00004  1.023323 -2.133468  1.242266  0.939837 -0.264515 -1.792044   

        x_6       x_7       x_8       x_9      x_10          y  
0  0.470004 -0.139467  0.623996  0.320359  0.078612  83.424500  
1  0.503860 -0.219545  0.697607  0.238306  0.081778  79.374109  
2  0.498045 -0.194247  0.684134  0.259392  0.095003  82.181616  
3  0.503359 -0.201923  0.686394  0.245736  0.091737  83.006586  
4  0.470478 -0.142896  0.623778  0.314610  0.078987  83.051434

In [2]:
train_X = train.drop(columns=['ID','y'])
train_y = train['y']

train_data = pd.concat([train_X, train_y], axis=1)

In [3]:
# normalize
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

train_X_nor = minmax.fit_transform(train_X)

train_X_nor = pd.DataFrame(train_X_nor, columns=train_X.columns)

train_data_nor = pd.concat([train_X_nor, train_y.reset_index(drop=True)], axis=1) # 인덱스 불일치로 인한 결합 오류 방지

In [4]:
from pycaret.regression import setup, compare_models, pull, save_model, load_model, predict_model

regressor = setup(data = train_data, target='y', session_id=42)
best_model = compare_models()
results = pull()

results

Model     MAE     MSE    RMSE      R2  \
br                         Bayesian Ridge  1.2057  2.8778  1.6879  0.6054   
lr                      Linear Regression  1.2057  2.8779  1.6879  0.6053   
gbr           Gradient Boosting Regressor  1.2083  2.8857  1.6903  0.6043   
ridge                    Ridge Regression  1.2080  2.8875  1.6908  0.6040   
lightgbm  Light Gradient Boosting Machine  1.2128  2.9023  1.6953  0.6019   
huber                     Huber Regressor  1.2009  2.9292  1.7034  0.5982   
lar                Least Angle Regression  1.2256  2.9415  1.7070  0.5965   
et                  Extra Trees Regressor  1.2380  2.9828  1.7190  0.5907   
rf                Random Forest Regressor  1.2361  2.9976  1.7231  0.5888   
omp           Orthogonal Matching Pursuit  1.3052  3.2432  1.7936  0.5546   
knn                 K Neighbors Regressor  1.3282  3.4143  1.8408  0.5306   
ada                    AdaBoost Regressor  1.3526  3.5778  1.8858  0.5072   
dt                Decision Tree Regressor  1.7510  5.7934  2.4033  0.1986   
par          Passive Aggressive Regressor  1.8994  6.3477  2.3462  0.1342   
en                            Elastic Net  1.8033  6.9943  2.6428  0.0324   
lasso                    Lasso Regression  1.8369  7.2304  2.6871 -0.0005   
llar         Lasso Least Angle Regression  1.8369  7.2304  2.6871 -0.0005   
dummy                     Dummy Regressor  1.8369  7.2304  2.6871 -0.0005   

           RMSLE    MAPE  TT (Sec)  
br        0.0220  0.0146     0.022  
lr        0.0220  0.0146     0.365  
gbr       0.0220  0.0146     2.511  
ridge     0.0220  0.0146     0.010  
lightgbm  0.0221  0.0147     0.386  
huber     0.0221  0.0145     0.162  
lar       0.0222  0.0148     0.024  
et        0.0223  0.0150     1.383  
rf        0.0224  0.0150     5.601  
omp       0.0231  0.0158     0.028  
knn       0.0237  0.0161     0.024  
ada       0.0243  0.0164     0.624  
dt        0.0309  0.0211     0.120  
par       0.0293  0.0230     0.024  
en        0.0324  0.0216     0.016  
lasso     0.0329  0.0219     0.013  
llar      0.0329  0.0219     0.010  
dummy     0.0329  0.0219     0.011

In [5]:
save_model(best_model, 'best_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['x_0', 'x_1', 'x_2', 'x_3', 'x_4',
                                              'x_5', 'x_6', 'x_7', 'x_8', 'x_9',
                                              'x_10'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model', BayesianRidge())]),
 'best_model.pkl')

In [6]:
test_X = test.drop(columns=['ID'])
best_model = load_model('best_model')
preds = predict_model(best_model, data = test_X)

preds

Transformation Pipeline and Model Successfully Loaded


x_0       x_1       x_2       x_3       x_4       x_5       x_6  \
0     1.069219 -2.106844  1.228438  0.897236 -0.258323 -1.794707  0.467295   
1     1.071553 -2.509419  1.153271  0.824917 -0.270175 -1.773425  0.501530   
2     0.977364 -1.362481  1.164106  0.948528 -0.417688 -1.633350  0.517789   
3     0.977220 -1.211227  1.138849  0.914028 -0.459511 -1.602302  0.538293   
4     1.069111 -2.409491  1.174731  0.856938 -0.273046 -1.782787  0.504689   
...        ...       ...       ...       ...       ...       ...       ...   
4981  1.037452 -2.132522  1.236089  0.860538 -0.252378 -1.805769  0.472018   
4982  0.932432 -1.193839  1.139827  0.955182 -0.471994 -1.600613  0.541196   
4983  1.083483 -2.371390  1.214140  0.850023 -0.247225 -1.811753  0.481727   
4984  1.072450 -2.377752  1.234282  0.882019 -0.251468 -1.809643  0.479923   
4985  1.047948 -2.372358  1.192046  0.874905 -0.269210 -1.794239  0.496956   

           x_7       x_8       x_9      x_10  prediction_label  
0    -0.154325  0.630702  0.312034  0.084618         83.722534  
1    -0.203479  0.693976  0.246928  0.086857         82.149422  
2    -0.081149  0.581488  0.386941  0.002027         90.251816  
3    -0.069170  0.580191  0.397438 -0.017518         91.377396  
4    -0.197011  0.690465  0.246498  0.089634         82.232330  
...        ...       ...       ...       ...               ...  
4981 -0.158855  0.636707  0.302851  0.087101         83.245758  
4982 -0.071568  0.583131  0.397209 -0.017859         91.493858  
4983 -0.175863  0.661456  0.287690  0.094479         82.774963  
4984 -0.168748  0.667690  0.278757  0.096796         83.043510  
4985 -0.192248  0.692026  0.252981  0.092669         82.161377  

[4986 rows x 12 columns]

In [7]:
submission = pd.read_csv('../../data/sample_submission.csv')
submission['y'] = preds['prediction_label']
submission.to_csv('../normalized_AutoML/nor_AutoML_submission.csv',index=False)